In [18]:
import random
import faker
from datetime import datetime
import time

from azure.cosmos import CosmosClient
import azure.functions as func
import keys

client = CosmosClient(url=keys.COSMOS_ENDPOINT, credential=keys.COSMOS_KEY)
database = client.get_database_client("DBreviews")
container_users = database.get_container_client("user")
container_restaurants = database.get_container_client("restaurant")
container_reviews = database.get_container_client("review")

fake = faker.Faker()

positive_reviews = {
    "Italian": ["Loved the authentic pizza!", "Pasta was divine.", "Best tiramisu I've ever had!", "Incredible risotto.", "The gelato was out of this world."],
    "Chinese": ["The dim sum was exceptional!", "Loved the Peking duck.", "Best hot pot experience.", "Amazing dumplings!", "Crispy sweet and sour pork."],
    "Mexican": ["Fantastic tacos!", "Delicious enchiladas.", "Great guacamole.", "Perfect burritos.", "Authentic quesadillas."],
    "Indian": ["Butter chicken was heavenly.", "Amazing biryani.", "Delicious naan bread.", "Tasty tandoori chicken.", "Incredible masala chai."],
    "Japanese": ["Sushi was top-notch.", "Great ramen, very authentic.", "Loved the tempura.", "Delicious teriyaki chicken.", "Best miso soup I've tasted."]
}

neutral_reviews = {
    "Italian": ["Pizza was okay, not great.", "Pasta was average.", "Tiramisu was alright.", "Risotto was mediocre.", "Gelato was decent."],
    "Chinese": ["Dim sum was just fine.", "Peking duck was okay.", "Hot pot was nothing special.", "Dumplings were average.", "Sweet and sour pork was standard."],
    "Mexican": ["Tacos were alright.", "Enchiladas were passable.", "Guacamole was average.", "Burritos were okay.", "Quesadillas were so-so."],
    "Indian": ["Butter chicken was decent.", "Biryani was unremarkable.", "Naan bread was okay.", "Tandoori chicken was average.", "Masala chai was fair."],
    "Japanese": ["Sushi was ordinary.", "Ramen was passable.", "Tempura was average.", "Teriyaki chicken was okay.", "Miso soup was standard."]
}

negative_reviews = {
    "Italian": ["Pizza was undercooked.", "Pasta was too salty.", "Tiramisu lacked flavor.", "Risotto was overcooked.", "Gelato tasted bland."],
    "Chinese": ["Dim sum was disappointing.", "Peking duck was dry.", "Hot pot was bland.", "Dumplings were greasy.", "Sweet and sour pork was soggy."],
    "Mexican": ["Tacos were too spicy.", "Enchiladas were dry.", "Guacamole was tasteless.", "Burritos were soggy.", "Quesadillas were burnt."],
    "Indian": ["Butter chicken was too creamy.", "Biryani was overcooked.", "Naan bread was stale.", "Tandoori chicken was dry.", "Masala chai was weak."],
    "Japanese": ["Sushi was not fresh.", "Ramen was too salty.", "Tempura was oily.", "Teriyaki chicken was burnt.", "Miso soup was watery."]
}
restaurants = [
    {"id": "restaurant_001", "name": "Gusto's Italian", "location": "123 Main St", "cuisine": "Italian", "operational_hours": "9 AM - 9 PM"},
    {"id": "restaurant_002", "name": "Beijing Bites", "location": "456 Oak Ave", "cuisine": "Chinese", "operational_hours": "9 AM - 9 PM"},
    {"id": "restaurant_003", "name": "Carlos Mexican", "location": "Sreet Wallet JT", "cuisine": "Mexican", "operational_hours": "9 AM - 9 PM"},
     {"id": "restaurant_004", "name": "Zanda BAHABI", "location": "Eve Ball St", "cuisine": "Indian", "operational_hours": "9 AM - 9 PM"},
      {"id": "restaurant_005", "name": "Chu Wawa", "location": "Santas Balmas", "cuisine": "Indian", "operational_hours": "9 AM - 9 PM"}
]

def generate_user():
    return {
        "id": fake.uuid4(),
        "name": fake.name(),
        "email": fake.email(),
        "location": fake.city(),
        "preferences": random.choice(["Vegan", "Seafood", "Spicy", "Desserts", "Vegetarian"]),
        "dob": fake.date_of_birth().strftime("%Y-%m-%d")

    }


def generate_review(user_id, restaurant):
    rating = random.randint(1, 5)
    cuisine = restaurant["cuisine"]

    if rating >= 4:
        review_text = random.choice(positive_reviews[cuisine])
    elif rating == 3:
        review_text = random.choice(neutral_reviews[cuisine])
    else:
        review_text = random.choice(negative_reviews[cuisine])

    return {
        "id": fake.uuid4(),
        "restaurant_id": restaurant["id"],
        "user_id": user_id,
        "rating": rating,
        "review_text": review_text,
        "review_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

users = [generate_user() for _ in range(10)]

try:
    while True:
        user = random.choice(users)
        restaurant = random.choice(restaurants)
        review = generate_review(user["id"], restaurant)

        try:
            container_users.create_item(user)
        except Exception as e:
            pass

        try:
            container_restaurants.create_item(restaurant)
        except Exception as e:
            pass

        try:
            container_reviews.create_item(review)
        except Exception as e:
            pass

        print(review)
        print(user)
        print(restaurant)
        time.sleep(5)

except KeyboardInterrupt:
    print("La génération de données en temps réel a été arrêtée.")


{'id': '277a5590-4eae-4da1-a939-ed385fc0136b', 'restaurant_id': 'restaurant_004', 'user_id': 'd660cfd4-4246-4517-9680-6c3875504f08', 'rating': 3, 'review_text': 'Tandoori chicken was average.', 'review_date': '2023-12-26 11:48:06'}
{'id': 'd660cfd4-4246-4517-9680-6c3875504f08', 'name': 'Katelyn Estrada', 'email': 'joseph52@example.com', 'location': 'Bennettview', 'preferences': 'Vegan', 'dob': '2004-09-11'}
{'id': 'restaurant_004', 'name': 'Zanda BAHABI', 'location': 'Eve Ball St', 'cuisine': 'Indian', 'operational_hours': '9 AM - 9 PM'}
{'id': '7a261ba5-39b1-4e29-9a5c-7e959d2984ee', 'restaurant_id': 'restaurant_002', 'user_id': 'e990da66-8a81-45a3-9d25-ffc74dd3fc28', 'rating': 5, 'review_text': 'Crispy sweet and sour pork.', 'review_date': '2023-12-26 11:48:18'}
{'id': 'e990da66-8a81-45a3-9d25-ffc74dd3fc28', 'name': 'Jessica Patel', 'email': 'meghanmunoz@example.net', 'location': 'South Diana', 'preferences': 'Spicy', 'dob': '1982-09-20'}
{'id': 'restaurant_002', 'name': 'Beijing Bite